In [ ]:
import json

# pass.json stores the data for each iteration of our
# optimiser which created the reactor configuration we 
# expected.
with open("pass.json", "r") as f:
    pass_array = json.load(f)

# fail.json stores the data for each iteration of our
# optimiser which created the reactor configuration we 
# did not expect.
with open("fail.json", "r") as f:
    fail_array = json.load(f)

# states of the optimiser when we start to see solution divergence
pass6 = pass_array[5]
fail6 = fail_array[5]

In [ ]:
import numpy as np

# show both that we have the same initial state...
assert pass6["f"] == fail6["f"]
assert np.array_equal(np.array(pass6["x"]), np.array(fail6["x"]))
assert np.array_equal(np.array(pass6["B"]), np.array(fail6["B"]))
assert np.array_equal(np.array(pass6["df"]), np.array(fail6["df"]))
assert np.array_equal(np.array(pass6["deq"]), np.array(fail6["deq"]))
assert np.array_equal(np.array(pass6["eq"]), np.array(fail6["eq"]))
assert np.array_equal(np.array(pass6["lbs"]), np.array(fail6["lbs"]))
assert np.array_equal(np.array(pass6["ubs"]), np.array(fail6["ubs"]))

In [ ]:
# ... but get two different solutions to the qsp
if not np.array_equal(np.array(pass6["delta"]), np.array(fail6["delta"])):
    print("Delta arrays NOT the same")
else:
    print("Delta arrays the same")

In [ ]:
f = pass6["f"]
x = np.array(pass6["x"])
B = np.array(pass6["B"])
df = np.array(pass6["df"])
deq = np.array(pass6["deq"])
eq = np.array(pass6["eq"])
lbs = np.array(pass6["lbs"])
ubs = np.array(pass6["ubs"])

In [ ]:
import cvxpy as cp

# np.random.seed(42)

solutions = []

for i in range(100):
    # seeding doesn't seem to remove this non-determinism :(
    np.random.seed(42)

    # recreate `solve_qsp` in https://github.com/ukaea/PyVMCON/blob/main/src/pyvmcon/vmcon.py#L184C4-L184C4
    # where we are observing this non-determinism.
    
    delta = cp.Variable(x.shape)
    problem_statement = cp.Minimize(
        f
        + (0.5 * cp.quad_form(delta, B, assume_PSD=True))
        + (delta.T @ df)
    )

    constraints = []
    constraints.append(x + delta >= lbs)
    constraints.append(x + delta <= ubs)
    constraints.append((deq @ delta) + eq == 0)

    qsp = cp.Problem(problem_statement, constraints or None)
    qsp.solve(verbose=True, solver=cp.OSQP, eps_rel=1e-1)

    for e in solutions:
        if np.array_equal(delta.value, e):
            break
    else:
        solutions.append(delta.value)

# show flip-flop between solution seen in the 'pass' state
# and solution seen in the 'fail' state.

# Run this cell multiple times to see the proportion of `equals_pass6`
# and `equals_fail6` changes. I have never personally observed any 
# `other` solutions.
print(f"{len(solutions) = }")